<div style="text-align:right"><font size=7 color="orchid" face='Brush Script MT'> to Start, Click - <button class="btn btn-sm btn-success"><i class="fa fa-lg fa-id-card-o">

<font color="magenta" size=7><i>CPG Simulation</i></font>
    
<font color="gray"><i>A small purpose built neural network uses proprioceptive feedback to control a modular robot

**<div class="text-center">Abstract</div>**
We propose a modular architecture for neuromorphic closed-loop control based on bistable relaxation oscillator modules consisting of three spiking neurons each. Like its biological prototypes, this basic component is robust to parameter variation but can be modulated by external inputs. By combining these modules, we can construct a neural state machine capable of generating the cyclic or repetitive behaviors necessary for legged locomotion. A concrete case study for the approach is provided by a modular robot constructed from flexible plastic volumetric pixels, in which we produce a forward crawling gait entrained to the natural frequency of the robot by a minimal system of twelve neurons organized into four modules.


<div class="text-center"><a href="https://doi.org/10.1371/journal.pone.0240267">Paper Link</a></div>
    

    
**Authors** Alex Spaeth, Maryam Tebyani, David Haussler, Mircea Teodorescu

In [ ]:
from IPython.core.display import HTML,Image
import ipywidgets as ipw   #HTML("""<h2 class='text-center text-danger'>Tutorial coming Soon</h2><div class="row"><div class="col-xs-12 col-md-offset-3 col-md-6"><div class="embed-responsive embed-responsive-16by9"><iframe class="embed-responsive-item"src='https://www.youtube.com/embed/pwBs4J4xDOw'></iframe></div></div></div>""")
HTML("""<div class="row"><div class="col-xs-12"><div class="embed-responsive embed-responsive-16by9"><iframe class="embed-responsive-item"src='https://www.youtube.com/embed/3h20uhEVuB0'></iframe></div></div></div>""")


# Neuron Simulation Code

This section contains the parameters and code for simulation of the  neuron model. The dictionaries RS and LTS describe single-neuron parameters, and then there are a bunch of different methods producing parameter arrays suitable for the simulation code.

The simulation code is in the function `solve_neurons`, which can perform a simulation until a fixed termination time or until steady state. It requires a parameter array µ, a synaptic connectivity matrix G, and an initial condition x0. The docstring describes the format of these parameters in more detail.

``` julia

using LinearAlgebra
using Statistics
using MultivariateStats
using RecursiveArrayTools

import DifferentialEquations
DE = DifferentialEquations

import PyPlot
Plt = PyPlot.plt
```

``` julia
"Parameters for regular-spiking (RS) neuron."
const RS = Dict(:a => 0.03, :b => -2.0, :c => -50.0, :d => 100.0, :C => 100.0, 
    :k => 0.7, :Vr => -60.0, :Vt => -40.0, :Vn => 0.0, :Vp => 35.0, :τ => 5.0)
"Parameters for low-threshold-spiking (LTS) interneuron."
const LTS = Dict(:a => 0.03, :b => 8.0, :c => -53.0, :d => 20.0, :C => 100.0,
    :k => 1.0, :Vr => -56.0, :Vt => -42.0, :Vn => -70.0, :Vp => 20.0, :τ => 20.0)

"Excitatory connection between two neurons of a single module."
const G_exc = 20.0
"Excitatory connection keeping the reset input easy to activate."
const G_rst = 5.0
"Inhibitory connection from the reset input of the module."
const G_inh = 10.0
"Feedforward connection for gradual activation of other modules."
const G_ffw = 10.0
"Feedback connection for deactivation of other modules."
const G_fb = 10.0


"""
    Gij(; G_exc, G_inh, G_rst)

Constructs a connectivity matrix for a single module given the three 
defining conductances.
"""
Gij(; G_exc=G_exc, G_inh=G_inh, G_rst=G_rst) = [0.0  G_exc G_inh;
                                                G_exc  0.0  G_inh;
                                                G_rst G_rst  0.0 ]

"""
    x0_fire1(μ)

Given a parameter array μ, returns an initial condition just before a
firing of the first cell.
"""
function x0_fire1(μ=μ_module)
  Vr = μ.x[7]
  Vp = μ.x[10]
  d = μ.x[4]
  ArrayPartition([Vp[1]; Vr[2:end]],
                 zero(d),
                 zeros(length(d)),
                 zeros(length(d)))
end


"All the names of the neuron parameters."
parameter_names = (:a, :b, :c, :d, :C, :k, :Vr, :Vt, :Vn, :Vp, :τ)


"""
    μ_from_types(types)

Assemble a parameter array from a list of neuron types.
"""
μ_from_types(types...) = ArrayPartition([[t[param] for t in types]
                                         for param in parameter_names]...)

"Default parameter array for a module: two RS neurons and one LTS."
μ_module = μ_from_types(RS, RS, LTS)
```

``` julia 
"""
    jump_neuron!(i, x, μ)

Given a state, perform the reset for the ith neuron.
"""
function jump_neuron!(i, x, μ)
  x.x[1][i] = μ.x[3][i]
  x.x[2][i] += μ.x[4][i]
  x.x[4][i] += 1.0
  x
end


"""
    jump_map(μ, x)

Given a state, return a copy with all applicable spike resets applied.
"""
function jump_map(μ, x)
  x = copy(x)
  for (i,dv) in enumerate(x.x[1] .- μ.x[10])
    if dv >= 0
      jump_neuron!(i, x, μ)
    end
  end

  x
end


"""
    solve_neurons(μ, G, x0)

Integrate the forward dynamics of a system of N neurons. If a finite
stop time tmax is provided, integrates until that point. Otherwise,
integrates until the first spike of a cell listed in termination_indices,
or until the system reaches a steady state.

The neuron parameters are stored in an ArrayPartition μ with one
component per parameter, containing N parameter values. The synaptic
connectivity matrix G is just an N×N array. The initial condition x0
should be an ArrayPartition with four components corresponding to the
per-neuron initial values of v, u, x, and y.
"""
function solve_neurons(μ, G, x0; tmax=Inf, fp_tol=1e-4,
                       termination_indices=[], solver_args...)

  # The in-place continuous dynamics of the neurons.
  function izh(dx, x, (μ,G), t)
    a,b,c,d, C,k,vr,vt,vn,vp, τ = μ.x

    sodium = @.k*(x.x[1] - vr)*(x.x[1] - vt)
    Isyn = G*(x.x[3] .* vn)  .-  (G*x.x[3]) .* x.x[1]
    dx.x[1] .= @. (sodium - x.x[2] + Isyn)/C
    dx.x[2] .= @. a*(b*(x.x[1]-vr) - x.x[2])
    dx.x[3] .= @. x.x[4] / τ
    dx.x[4] .= @. -(x.x[3] + 2x.x[4]) / τ
  end


  # This is the callback that produces the spiking behavior. It is
  # vectorized and done in-place for efficiency. The first function
  # condition() checks whether any voltage exceeds the corresponding
  # cell's Vp, and the second function affect!() does the reset for
  # that cell (or terminates if it's on the termination list).
  function condition(out, u,t,integrator)
    out .= u.x[1] .- integrator.p[1].x[10]
  end

  function affect!(integrator, i)
    if i in termination_indices
      integrator.u.x[1][i] = integrator.p[1].x[10][i]
      DE.terminate!(integrator)
    else
      jump_neuron!(i, integrator.u, integrator.p[1])
    end
  end

  # The VectorContinuousCallback needs to know how many callbacks
  # there are, which is the number of neurons. Get that information
  # from the number of voltage values in the initial condition.
  spike = DE.VectorContinuousCallback(condition, affect!, length(x0.x[1]))

  # Handle infinite tmax by waiting for steady state. This code is
  # taken from an example in the DifferentialEquations.jl docs and
  # modified to set the time to infinity before terminating so it's
  # clear that we've reached a steady-state solution.
  callbacks = if tmax != Inf
    spike
  else
    function allDerivPass(integrator, abstol, reltol)
      if DE.isinplace(integrator.sol.prob)
        testval = first(DE.get_tmp_cache(integrator))
        DE.get_du!(testval, integrator)
      else
        testval = get_du(integrator)
      end

      !any(abs(d) > abstol && abs(d) > reltol*abs(u)
           for (d,abstol, reltol, u) =
           zip(testval, Iterators.cycle(abstol),
               Iterators.cycle(reltol), integrator.u))
    end

    at_steady_state = (u, t, integrator) -> allDerivPass(integrator,
                                                         fp_tol, fp_tol)
    function terminate_steady_state!(integrator)
      DE.set_t!(integrator, Inf)
      DE.terminate!(integrator)
    end
    tss = DE.DiscreteCallback(at_steady_state,
                              terminate_steady_state!;
                              save_positions = (true, false))
    DE.CallbackSet(spike, tss)
  end

  # Solve the problem and return it!
  prob = DE.ODEProblem(izh, x0, (0.0, tmax), (μ,G))
  DE.solve(prob, DE.Tsit5(), callback=callbacks; solver_args...)
end
```

# Mathematical Analysis Code

This section of the notebook defines the functions that are used for the mathematical analysis of the neuron simulations that can be run using the above code. In particular, we define the Poincaré map of a neural system along the Poincaré section $V_1 = V_p$ as well as functions which find the fixed point and the corresponding return time, and all the various methods for random variations of neuron types that will be required for the Monte Carlo analysis later.

``` julia
"""
    poincaré(μ, G, x0)

For an initial condition on the Poincaré section, perform a jump
and compute the first return time and location in phase space.
Returns infinite time if we never return to the Poincaré section
due to finding a fixed point in the dynamics.
"""
function poincaré(μ, G, x0; solver_args...)
  sol = solve_neurons(μ, G, jump_map(μ, x0);
                      termination_indices=(1,),
                      save_everystep=false, solver_args...)
  sol.t[end], sol[end]
end


"""
    period_and_fp(μ, G)

For a parameter set μ and a connectivity matrix G, find the fixed
point of the Poincaré map and return the corresponding first return
time. This is done through simple Picard fixed-point iteration.
"""
function period_and_fp(μ, G, x0=nothing; maxiter=500, warn=true,
                       atol=1e-1, rtol=1e-3, pc_args...)

  if x0 === nothing
    x0 = x0_fire1(μ)
  end

  for iter in 1:maxiter
    xprev = x0
    T,x0 = poincaré(μ, G, x0; pc_args...)

    if isapprox(x0, xprev; atol=atol, rtol=rtol)
      return T==Inf ? 0 : T, x0
    end
  end

  if warn
    @warn "Convergence failed after $(maxiter) iterations."
  end
  return Inf, x0
end


"""
    modify_type(type; kw...)

Take a neuron type and return a new type where all the keywords in kw
have been applied to replace the corresponding parameters.
"""
modify_type(type; kw...) = merge(type, kw)


"""
    μ_mod1(;kw...)

Modifies the RS type of  the first neuron in a module.
"""
μ_mod1(; kw...) = μ_from_types(modify_type(RS; kw...), RS, LTS)


"""
    μ_modsame(;kw...)

Modifies the RS type of  the first neuron in a module.
"""
μ_modsame(; kw...) = let type = modify_type(RS; kw...)
  μ_from_types(type, type, LTS)
end


"""
    randomize_type(type, frac; kw...)

Take a neuron type and an amount of fractional variation, then apply
that amount of variation to each of the parameters of the type. If a
keyword argument is given, this is used as an absolute radius of
variation instead, in order to handle parameters equal to zero. Return
the applied fractional variation ξ as well as the actual parameters as
two separate dictionaries.
"""
function randomize_type(type, frac; kw...)
  ξ = Dict(zip(keys(type), randsphere(length(type), frac)))
  Dict(k => v + get(kw, k, v)*ξ[k] for (k,v) in type), ξ
end


"""
    module_works(μ, G; [Tmin], [x0])

Quantify whether a module works: the default initial condition must be
within the basin of attraction of a spiking limit cycle with period at
least equal to the given minimum period.
"""
function module_works(μ, G; Tmin=0.0, x0=nothing, fp_args...)
  Tmin < period_and_fp(μ, G, x0; fp_args...)[1] < Inf
end


"""
    randsphere(d, r=1)

Generate points uniformly at random on the radius-r (d-1)-sphere in ℝᵈ.
"""
randsphere(d, r=1) = let gaussblob = randn(d)
  r .* gaussblob ./ norm(gaussblob)
end


"""
    randball(d, r=1)

Generate points uniformly at random in the d-ball of radius r.
"""
randball(d, r=1) = randsphere(d,r) .* (rand() ^ (1/d))


"""
    μ_rand1(frac)

Generate a parameter array corresponding to random variation in the
parameters of the first neuron, returning also the fractional change
ξ. Variation in the excitatory synaptic reversal potential is absolute
and multiplied by 100 mV because its default value is exactly zero.
"""
function μ_rand1(frac)
  type, ξ = randomize_type(RS, frac; Vn=100)
  μ_from_types(type, RS, LTS), ξ
end


"""
    μ_randsame(frac)

Generate a parameter array corresponding to random variation in the
parameters of the excitatory neurons, with both cells modulated in
exactly the same way. Also returns the modification applied.
"""
function μ_randsame(frac)
  type, ξ = randomize_type(RS, frac; Vn=100)
  μ_from_types(type, type, LTS), ξ
end


"""
    μ_rand2(frac)

Generate a parameter array corresponding to random variation in the
parameters of the excitatory neurons, returning also the fractional
changes ξ1 and ξ2.
"""
function μ_rand2(frac)
  type1, ξ1 = randomize_type(RS, frac; Vn=100)
  type2, ξ2 = randomize_type(RS, frac; Vn=100)
  μ_from_types(type1, type2, LTS), ξ1, ξ2
end
```

# Figure 2: Module Electrical Activity Demo

This figure introduces a bit of extra synaptic input to the inhibitory interneuron within the module in order to cause it to deactivate the module after a short period of activity. The synaptic matrix is the same as the default connectivity matrix for the module, but with a stronger connection from the excitatory cells of the module to the reset interneuron. 

``` julia
μ, G = μ_module, Gij(; G_rst=7.0747)

sol = solve_neurons(μ, G, jump_map(μ, x0_fire1(μ)); tmax=1000)

fig, axes = Plt.subplots(2,1)

t = range(0,150; length=1000)
axes[1].plot(t, [u.x[1] for u in sol(t)])
axes[2].plot(t, [G*(u.x[3] .* μ.x[9]) .- (G*u.x[3]) .* u.x[1]
                    for u in sol(t)])

for ax in axes
    ax.set_xticks([])
    ax.set_yticks([])
    for side in [:top, :bottom, :left, :right]
        ax.spines[string(side)].set_visible(false)
    end
end

axes[1].set_ylabel("Membrane Voltage")
axes[end].set_ylabel("Synaptic Input")
fig.legend(("E1", "E2", "I"); ncol=3, loc="lower center")

# fig.savefig("Fig2.tif")
```

![alt text](figures/Fig2.png "Title")

# Figure 3: Effect of Parameter Variation

``` julia
fig, axs = Plt.subplots(4,1)

t = range(1,100; length=1000)

for ax in axs
    μ = if ax === axs[1] 
        μ_module
    else
        μ_rand2(0.1)[1]
    end
    
    sol = solve_neurons(μ, Gij(), jump_map(μ, x0_fire1(μ)); tmax=2e2)
    ax.plot(t, [u.x[1][1:2] for u in sol(t.+100)])
    
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim(0,100)
    
    for side in ["top", "bottom", "left", "right"]
        ax.spines[side].set_visible(false)
    end
end

fig.legend(axs[end].lines, [raw"$E_1$", raw"$E_2$"]; 
    ncol=3, loc="lower right")
axs[end].set_xticks([])
axs[end].set_xlabel("")

fig.text(0.05, 0.5, "Membrane Voltage"; rotation="vertical", va="center", size=8)
# fig.savefig("Fig3.tif")
```

![alt text](figures/Fig3.png "Title")

# Method of Bisection

We determine the exact point where a parameter leaves its acceptable range using the method of bisection. This is implemented in the following relatively general-purpose code.

``` julia
"""
    bisect(P, a, b; atol=0, rtol=√ε)

Uses the bisection method to narrow an interval where some predicate
changes sign until its endpoints are equal with relative precision
`rtol` and absolute precision `atol`. Alternately, if the optional
argument `sigfigs` is provided, the tolerance is set to produce a
result accurate to that many significant figures.
"""
function bisect(P, a, b; sigfigs=nothing, kw...)
  Pa, Pb = P(a), P(b)
  @assert(Pa != Pb, "Truth value of predicate must differ at bounds.")

  if sigfigs !== nothing
    kw = (rtol=10.0^-sigfigs,
          atol=abs(a-b)*10.0^-sigfigs)
  end

  while !isapprox(a,b; kw...)
    guess = (a+b)/2

    if P(guess) == Pa
      a = guess
    else
      b = guess
    end
  end

  a, b
end


"""
    refine_bound(P, x0, xguess; rtol, atol)

For a predicate true at x0 but false at xguess, refine the bound by
bisection.
"""
function refine_bound(P, x0, xguess; sigfigs=nothing, kw...)
  x = bisect(P, x0, xguess; sigfigs, kw...)[1]

  if sigfigs === nothing
    x
  else
    round(x; sigdigits=sigfigs)
  end
end


"""
    refine_bounds(P, x0, xlow, xhigh; rtol, atol)

For a predicate true at x0 but false and xlow and xhigh,
simultaneously refine both bounds by bisection.
"""
function refine_bounds(P, x0, xlow, xhigh; kw...)
  (refine_bound(P, x0, xlow; kw...),
   refine_bound(P, x0, xhigh; kw...))
end
```

# Acceptable Parameter Ranges

We can compute the acceptable range of variation in each single-neuron parameter by checking whether the spiking limit cycle continues to exist as described in the text. These all follow the same pattern of ensuring that the standard initial condition leads to an attractive fixed point of the Poincaré map. We can also do exactly the same thing, mathematically speaking, with the single-module synaptic conductance parameters $G_\text{exc}$ and $G_\text{rst}$: they both cease to function when the spiking limit cycle disappears.

On the other hand, as described in the text, the simplest way to check that a value of $G_\text{inh}$ is acceptable is to increase $G_\text{rst}$ enough that persistent activity causes the reset interneuron to fire; if this breaks the limit cycle and returns the neuron to a resting state, the reset strength is sufficient. As a result, we check that perturbed values of $G_\text{inh}$ function correctly by checking that they are not acceptable in conjunction with a perturbed value of $G_\text{rst}$.

``` julia
a_bound = refine_bound(RS[:a], 0.001; sigfigs=3) do a
    module_works(μ_modsame(a=a), Gij(); Tmin=5.0, warn=false)
end

b_bound = refine_bound(RS[:b], 10*RS[:b]; sigfigs=3) do b
    module_works(μ_modsame(b=b), Gij(); Tmin=5.0, warn=false)
end

c_bound = refine_bound(RS[:c], 0; sigfigs=3) do c
    module_works(μ_modsame(c=c), Gij(); Tmin=5.0, warn=false)
end

d_bounds = refine_bounds(RS[:d], -RS[:d], 10*RS[:d]; sigfigs=3) do d
    module_works(μ_modsame(d=d), Gij(); Tmin=5.0, warn=false)
end

C_bounds = refine_bounds(RS[:C], 0.1*RS[:C], 10*RS[:C]; sigfigs=3) do C
    module_works(μ_modsame(C=C), Gij(); Tmin=5.0, warn=false)
end

k_bound = refine_bound(RS[:k], 10*RS[:k]; sigfigs=3) do k
    module_works(μ_modsame(k=k), Gij(); Tmin=5.0, warn=false)
end

Vr_bounds = refine_bounds(RS[:Vr], 10*RS[:Vr], -20.0; sigfigs=3) do Vr
    module_works(μ_modsame(Vr=Vr), Gij(); Tmin=5.0, warn=false)
end

Vt_bounds = refine_bounds(RS[:Vt], 10*RS[:Vt], -20.0; sigfigs=3) do Vt
    module_works(μ_modsame(Vt=Vt), Gij(); Tmin=5.0, warn=false)
end

Vp_bound = refine_bound(RS[:Vp], RS[:Vt]; sigfigs=3) do Vp
    module_works(μ_modsame(Vp=Vp), Gij(); Tmin=5.0, warn=false)
end

Vn_bounds = refine_bounds(RS[:Vn], RS[:Vt], 10.0; sigfigs=3) do Vn
    module_works(μ_modsame(Vn=Vn), Gij(); Tmin=5.0, warn=false)
end

τ_bounds = refine_bounds(RS[:τ], 1.0, 10.0; sigfigs=3) do τ
    module_works(μ_modsame(τ=τ), Gij(); Tmin=5.0, warn=false)
end

Gexc_bounds = refine_bounds(20.0, 15.0, 200.0; sigfigs=3) do G_exc
    module_works(μ_module, Gij(; G_exc); Tmin=5.0, warn=false)
end

Grst_bound = refine_bound(5.0, 10.0; sigfigs=3) do G_rst
    module_works(μ_module, Gij(; G_rst); Tmin=5.0, warn=false)
end

Ginh_bound = refine_bound(10.0, 0.0; sigfigs=3) do G_inh
    !module_works(μ_module, Gij(; G_inh, G_rst=10.0); Tmin=5.0, warn=false)
end

println("Changing both cells equally, the module still works for...")
println("    a > $a_bound")
println("    b > $b_bound")
println("    c < $c_bound")
println("    d ∈ $d_bounds")
println("    C ∈ $C_bounds")
println("    k < $k_bound")
println("   Vᵣ ∈ $Vr_bounds")
println("   Vₜ ∈ $Vt_bounds")
println("   Vₚ > $Vp_bound")
println("   Vₙ ∈ $Vn_bounds")
println("    τ ∈ $τ_bounds")
println(" Gexc ∈ $Gexc_bounds")
println(" Grst < $Grst_bound")
println(" Ginh > $Ginh_bound")
```

```
Changing both cells equally, the module still works for...
    a > 0.0171
    b > -6.08
    c < -40.3
    d ∈ (40.4, 169.0)
    C ∈ (68.3, 159.0)
    k < 1.41
   Vᵣ ∈ (-66.3, -29.2)
   Vₜ ∈ (-47.0, -36.2)
   Vₚ > -32.7
   Vₙ ∈ (-9.69, 9.67)
    τ ∈ (3.77, 7.41)
 Gexc ∈ (16.1, 31.6)
 Grst < 7.07
 Ginh > 4.37
 ```

# Monte Carlo Simulation

In this section, we set up and execute Monte Carlo simulations to verify that the majority of CPG modifications are harmless.

``` julia
"The number of CPG realizations to test for computing statistics."
N_montecarlo = 1000000

"""
    try_many(sample, N)

Try N different random variants of the module, recording both the
random modification to the parameters and whether the module still
works under those conditions.
"""
function try_many(sample, N)

  ξs = zeros(N, length(sample()[2]))
  worksp = falses(N)

  Threads.@threads for i in 1:N
    μ, ξs[i,:] = sample()
    worksp[i] = module_works(μ, Gij(); warn=false)
  end
  ξs, worksp
end
```


``` julia
@time ξs1, worksp1 = try_many(N_montecarlo) do
    μ, ξ = μ_rand1(0.10)
    μ, [ξ[p] for p in parameter_names]
end

println("Varying one neuron, modules break ", 
    round(100*(1 - mean(worksp1)); digits=1), "% of the time.")
```

```
848.728239 seconds (15.74 G allocations: 1.465 TiB, 63.61% gc time)
Varying one neuron, modules break 1.3% of the time.
```

``` julia
@time ξssame, workspsame = try_many(N_montecarlo) do
    μ, ξ = μ_randsame(0.10)
    μ, [ξ[p] for p in parameter_names]
end

println("Varying both neurons identically, modules break ", 
    round(100*(1 - mean(workspsame)); digits=1), "% of the time.")
```

```
875.569545 seconds (15.55 G allocations: 1.447 TiB, 63.08% gc time)
Varying both neurons identically, modules break 2.8% of the time.
```

``` julia
@time ξs2, worksp2 = try_many(N_montecarlo) do
    μ, ξ1, ξ2 = μ_rand2(0.10)
    μ, hcat([ξ1[p] for p in parameter_names],
            [ξ2[p] for p in parameter_names])
end

println("Varying both neurons independently, modules break ", 
    round(100*(1 - mean(worksp2)); digits=1), "% of the time.")
```

```
909.190263 seconds (16.10 G allocations: 1.501 TiB, 63.31% gc time)
Varying both neurons independently, modules break 5.4% of the time.
```

# Figure 4: LDA Point Cloud

Here, we identify the linear discriminant axis as well as a semi-arbitrary orthogonal axis of a smaller version of the point cloud corresponding to simultaneous variation of both neurons' parameter vectors. The vector is printed out in order to show the most important components ($V_t$, $V_r$, and $V_n$), and then the point cloud projected onto those two axes is plotted.

``` julia
# Use LDA to find the most explanatory axis of the results from the Monte Carlo 
# simulation of random variation in both excitatory neurons' parameters.
Xp = permutedims(ξssame[workspsame, :])
Xn = permutedims(ξssame[ .! workspsame, :])
lda = fit(LinearDiscriminant, Xp, Xn)
firstaxis = normalize(lda.w)

# Project that away and find a second, then orthogonalize.
secondguess = try
    Xp2 = Xp .- (firstaxis' * Xp) .* firstaxis
    Xn2 = Xn .- (firstaxis' * Xn) .* firstaxis
    fit(LinearDiscriminant, Xp2, Xn2).w
catch
    randn(length(firstaxis))
end
secondaxis = normalize(secondguess .- firstaxis .* (firstaxis ⋅ secondguess))

# Print out the first axis to identify its main components. You can see that the 
# most important components (magnitude about 0.5) correspond to Vt, Vr, and Vn.
round.(firstaxis * 10)
```

``` julia
11-element Array{Float64,1}:
  1.0
  0.0
 -0.0
 -1.0
 -1.0
 -1.0
 -5.0
  6.0
  6.0
  0.0
  2.0
```

``` julia
# Choose 1000 new points to plot, with higher variance for illustrative purposes.
ξsub, workspsub = try_many(1000) do
    μ, ξ1 = μ_randsame(0.15)
    μ, [ξ1[p] for p in parameter_names]
end

# Construct a projection matrix and apply it to the new points.
projection = [firstaxis secondaxis]'
Xpproj = projection * permutedims(ξsub[workspsub, :])
Xnproj = projection * permutedims(ξsub[ .!workspsub, :])

# This part actually plots the point cloud.
fig = Plt.figure()
ax = fig.gca(aspect=:equal)
ax.plot(Xpproj[1,:], Xpproj[2,:], ".", ms=3)
ax.plot(Xnproj[1,:], Xnproj[2,:], ".", ms=3)

# And all the rest is just making it look prettier for the paper. :)
fig.legend(["Working", "Non-working"]; 
    loc="upper right", markerscale=2,
    bbox_to_anchor=(0.9, 0.95))
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)
ax.set_xticks([-0.1, 0.1])
ax.set_xticklabels([raw"-10\%", raw"+10\%"])
ax.set_xlabel("Primary Discriminant Axis")
ax.set_yticks([-0.1, 0.1])
ax.set_yticklabels([raw"-10\%", raw"+10\%"])
ax.set_ylabel("Secondary Discriminant Axis")
# fig.savefig("Fig4.tif"; bbox_inches=:tight)
```

![alt text](figures/Fig4.png "Title")

# Simulating a Custom CPG

This code implements the neural network corresponding to the full four-module CPG developed in section 4 of the paper. This CPG consists of twelve neurons making up four identical modules, plus four muscle cells used to control the actuators. (It would be eight, except that the inputs to a given actuator's flexor and the extensor of its complementary actuator are identical, leading to identical behavior, so they are not simulated separately. This is a meaningless and potentially premature optimization.)

``` julia
"""
    Gcpg(; G_fb, G_ffw, G_exc, G_inh, G_rst)

Generate a synaptic connectivity matrix for the full four-module CPG.
There are also four muscle cells, which receive a weak connection of
strength G_musc. The idea is that this should be sufficient to create
a deviation in membrane voltage of about 1 mV.
"""
function Gcpg(; G_fb=G_fb, G_ffw=G_ffw, G_musc=1.0, Gij_args...)
  # Four three-cell modules plus four muscles.
  G = zeros(16,16)
  # Generate four modules this way.
  G[1:12,1:12] .= kron(Matrix(1.0I, 4, 4), Gij(Gij_args...))
  G[1,11] = G[4,2] = G[7,5] = G[10,8] = G_fb
  G[3,4] = G[6,7] = G[9,10] = G[12,1] = G_ffw
  # Now add the forward connections to the muscles.
  G[13,2] = G[14,5] = G[15,8] = G[16,11] = G_musc
  # And return the thing!
  G
end

"Parameter array for the full CPG, including muscles."
μcpg = μ_from_types(RS, RS, LTS, RS, RS, LTS, RS, RS, LTS, RS, RS, LTS,
                    RS, RS, RS, RS)



"""
    solve_cpg(μ, G)

Inelegant copypasta of solve_neurons, plus some glue code to make it
simulate the simple mass-damper system which stands in for the robot.
"""
function solve_cpg(μ, G, p; tmax=Inf, fp_tol=1e-4, solver_args...)

  # The in-place continuous dynamics of the neurons and actuators.
  function izh(dx, x, (μ,G,p), t)
    a,b,c,d, C,k,vr,vt,vn,vp, τ = μ.x

    sodium = @.k*(x.x[1] - vr)*(x.x[1] - vt)
    Isyn = G*(x.x[3] .* vn)  .-  (G*x.x[3]) .* x.x[1]
    dx.x[1] .= @. (sodium - x.x[2] + Isyn)/C
    dx.x[2] .= @. a*(b*(x.x[1]-vr) - x.x[2])
    dx.x[3] .= @. x.x[4] / τ
    dx.x[4] .= @. -(x.x[3] + 2x.x[4]) / τ

    # Fortunately, actuator dynamics are super simple, except for the
    # mess of ensuring that the actuators stay in bounds. Just make
    # sure that the velocity and position don't increase when the
    # position is already at the upper bound, and don't decrease when
    # it's at the lower bound.
    activations = x.x[1][end-3:end] - circshift(x.x[1][end-3:end], 2)
    activations = clamp.(activations, -1.0, 1.0)
    dx.x[5] .= x.x[6]
    dx.x[6] .= @. (activations - p[2]*x.x[6]) / p[1]
    dx.x[5][@. (x.x[5] >= 1.0) & (x.x[6] >= 0.0)] .= 0.0
    dx.x[6][@. (x.x[5] >= 1.0) & (dx.x[6] >= 0.0)] .= 0.0
    dx.x[5][@. (x.x[5] <= 0.0) & (x.x[6] <= 0.0)] .= 0.0
    dx.x[6][@. (x.x[5] <= 0.0) & (dx.x[6] <= 0.0)] .= 0.0

    # Finally, here's the proprioceptive feedback.
    Ifb = -p[3] .* (1 .- circshift(x.x[5], 1) .+ circshift(x.x[5], -1))
    for i in 1:4
      dx.x[1][3i-2] += Ifb[i] / C[3i-2]
      dx.x[1][3i-1] += Ifb[i] / C[3i-1]
    end
  end


  # This is the callback that produces the spiking behavior. It is
  # vectorized and done in-place for efficiency. The first function
  # condition() checks whether any voltage exceeds the corresponding
  # cell's Vp, and the second function affect!() does the reset for
  # that cell (or terminates if it's on the termination list).
  function should_fire(out, u,t,integrator)
    out .= u.x[1] .- integrator.p[1].x[10]
  end

  function spike_reset!(integrator, i)
    jump_neuron!(i, integrator.u, integrator.p[1])
  end

  # The VectorContinuousCallback needs to know how many callbacks
  # there are, which is the number of neurons. Get that information
  # from the number of values given for an arbitrary parameter.
  spike = DE.VectorContinuousCallback(should_fire, spike_reset!,
                                      length(μ.x[1]))

  # Also add callbacks for the actuator bounds.
  function actuator_bounds(out, u,t,integrator)
    out[1:4] .= -u.x[5]
    out[5:8] .= u.x[5] .- 1.0
  end

  function actuator_reset!(integrator, i)
    if i > 4
      integrator.u.x[5][i-4] = 1.0
      integrator.u.x[6][i-4] = 0.0
    else
      integrator.u.x[5][i] = 0.0
      integrator.u.x[6][i] = 0.0
    end
  end

  # Only count this callback in the positive direction so we can get
  # away from the walls without causing an explosion.
  crash = DE.VectorContinuousCallback(actuator_bounds, actuator_reset!,
                                      nothing, 8)

  # An initial condition including the states of all the neurons in
  # the CPG as well as the four actuators' position and velocity.
  x0 = ArrayPartition(jump_map(μcpg, x0_fire1(μcpg)).x...,
                      0.5ones(4), zeros(4))

  # Solve the problem and return it!
  prob = DE.ODEProblem(izh, x0, (0.0, tmax), (μ,G,p))
  DE.solve(prob, DE.Tsit5(), callback=DE.CallbackSet(spike, crash);
           solver_args...)
end

```

# Figure 7: Effect of Feedback

Here, we compare two copies of the constructed CPG, with and without proprioceptive feedback, in order to demonstrate the inability of the open-loop system to locomote effectively even though it is a correct implementation of the desired state machine.

``` julia
@time ol = solve_cpg(μcpg, Gcpg(), (20e3,570.0,0.0); tmax=10000.0);
```

10.774645 seconds (40.95 M allocations: 2.329 GiB, 5.15% gc time)

``` julia
@time cl = solve_cpg(μcpg, Gcpg(), (20e3,570.0,25.0); tmax=10000.0);
```

2.185406 seconds (11.43 M allocations: 862.187 MiB, 5.88% gc time)

``` julia
fig, axes = Plt.subplots(10,1; figsize=(3,4.2))

for (i,ax) in enumerate(axes[1:4])
    ax.axhline(0; c="grey", lw=0.5)
    ax.plot(ol.t, [u.x[1][3i-2] for u in ol.u]; c=:C0)
    ax.plot(ol.t, [u.x[1][3i-1] for u in ol.u]; c=:C0)
    ax.plot(cl.t, [u.x[1][3i-2] for u in cl.u]; c=:C1)
    ax.plot(cl.t, [u.x[1][3i-1] for u in cl.u]; c=:C1)
end

# Plot actuator positions with scale lines, and save two lines to label in the legend.
line_ol = line_cl = nothing
for (i,ax) in enumerate(axes[6:9])
    ax.axhline(0.5; c="grey", lw=0.5)
    line_ol = ax.plot(ol.t, [u.x[5][i] for u in ol.u]; c=:C0)[1]
    line_cl = ax.plot(cl.t, [u.x[5][i] for u in cl.u]; c=:C1)[1]
    ax.set_ylim(0,1)
end

# Clean up all those annoying borders.
for ax in axes
    ax.set_xlim(3000, 8000)
    ax.set_xticks([])
    ax.set_yticks([])
    for side in ("left", "right", "top", "bottom")
        ax.spines[side].set_visible(false)
    end
end

# Freaky axis labels because there's no room to label them separately.
fig.text(0.05, 0.35, "Actuator Position", rotation=:vertical, va=:center, size=8)
fig.text(0.05, 0.73, "Membrane Voltage", rotation=:vertical, va=:center, size=8)

# Title and save it!
fig.legend([line_ol, line_cl], ["Open-Loop", "Closed-Loop"]; 
    loc="center right", bbox_to_anchor=(0.8, 0.45))
# fig.savefig("Fig7.tif"; bbox_inches=:tight)
```

![alt text](figures/Fig7.png "Title")